In [1]:
import guidance
import transformers
path = '/home/marcotcr/.cache/huggingface/hub/vicuna-13b'
model = transformers.AutoModelForCausalLM.from_pretrained(path, device_map='auto') # 'auto' will split equally, but GPUs 2-3 are slow
tokenizer = transformers.AutoTokenizer.from_pretrained(path, device_map='auto')
vicuna = guidance.llms.transformers.Vicuna(model=model, tokenizer=tokenizer) # use cuda for GPU if you have >27GB of VRAM

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
chatgpt = guidance.llms.OpenAI("gpt-3.5-turbo")


In [3]:
vicuna_system = 'A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions.'
chatgpt_system = 'You are a helpful assistant.'

In [ ]:

def random_polynomial(n_roots, low=1, high=100):
    roots = [np.random.randint(low, high) for _ in range(n_roots)]
    # unique roots only
    while len(set(roots)) != n_roots:
        roots = [np.random.randint(low, high) for _ in range(n_roots)]
    poly = np.polynomial.polynomial.Polynomial.fromroots(roots)
    a = poly.coef.copy()
    a = a[::-1]
    text= ''
    for i, coef in enumerate(a):
        if i == len(a) - 1:
            text += f'{coef}'
        else:
            text += f'{coef}x^{len(a) - i - 1} + '
    return roots, text
roots, poly = random_polynomial(1, low=-30, high=30)
poly, roots

('1.0x^1 + 8.0', [-8])

In [ ]:
chatgpt_roots = guidance('''{{#system~}}
{{system_prompt}}
{{~/system}}
{{#user~}}
Please find the roots of the following equation: {{equation}}
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions.
{{/user}}
{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''', llm=chatgpt, system_prompt=chatgpt_system)
chatgpt_roots(equation=poly)
# program = program(system_prompt=chatgpt_system, equation=poly)

Stop program system You are a helpful assistant. user Please find the roots of the following equation: 1.0x^1 + 8.0 
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions.
 assistant The given equation is 1.0x^1 + 8.0, which can be simplified to x + 8 = 0.
To find the roots, we need to solve for x.
Subtracting 8 from both sides, we get x = -8.
Therefore, the only root of the equation is x = -8.
Thus, ROOTS = [-8].

In [6]:
program = guidance('''{{#system~}}
{{system_prompt}}
{{~/system}}
{{#user~}}
Please find the roots of the following equation: {{equation}}
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions.
{{/user}}
{{#assistant~}}
{{gen 'reasoning' temperature=0}}
ROOTS = [{{gen 'answer' temperature=0}}]
{{~/assistant~}}''', llm=vicuna, system_prompt=vicuna_system)
program = program(equation=poly, silent=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:15                                                                                   │
│                                                                                                  │
│   12 {{gen 'reasoning' temperature=0}}                                                           │
│   13 ROOTS = [{{gen 'answer' temperature=0}}]                                                    │
│   14 {{~/assistant~}}''', llm=vicuna, system_prompt=vicuna_system)                               │
│ ❱ 15 program = program(equation=poly, silent=True)                                               │
│   16                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'poly' is not defined

In [ ]:
import re
polys = []
answers = []
chatgpt_answers = []
vicuna_answers = []
for _ in range(10):
    roots, poly = random_polynomial(1, low=-30, high=30)
    chatgpt_answers.append(chatgpt_roots(equation=poly)['answer'])
    vicuna_answers.append(vicuna_roots(equation=poly)['answer'])

roots = re.search(r'ROOTS = \[(.*)\]', program['answer'])
np.array(list(map(float, roots.group(1).split(', '))))

In [4]:
pages = [
{'title': 'Fish oil - Mayo Clinic',
'url': 'https://www.mayoclinic.org/drugs-supplements-fish-oil/art-20364810',
'text' : '''Overview
Fish oil is a dietary source of omega-3 fatty acids. Your body needs omega-3 fatty acids for many functions, from muscle activity to cell growth.

Omega-3 fatty acids are derived from food. They can't be manufactured in the body. Fish oil contains two omega-3s called docosahexaenoic acid (DHA) and eicosapentaenoic acid (EPA). Dietary sources of DHA and EPA are fatty fish, such as salmon, mackerel and trout, and shellfish, such as mussels, oysters and crabs. Some nuts, seeds and vegetable oils contain another omega-3 called alpha-linolenic acid (ALA).

Fish oil supplements come in liquid, capsule and pill form.

People take fish oil for its anti-inflammatory effects.

Evidence
Research on the use of fish oil for specific conditions shows:

- Heart disease. While research shows that people who eat dietary sources of fish oil at least twice a week have a lower risk of dying of heart disease, taking fish oil supplements seems to have little to no benefits to heart health.
- High blood pressure. Multiple studies report modest reductions in blood pressure in people who take fish oil supplements. There's some evidence that the beneficial effects of fish oil might be greater for people with moderate to severe high blood pressure than for those with mild blood pressure elevation.
- High triglycerides and cholesterol. There's strong evidence that omega-3 fatty acids can significantly reduce blood triglyceride levels. There also appears to be a slight improvement in high-density lipoprotein (HDL, or "good") cholesterol, although an increase in levels of low-density lipoprotein (LDL, or "bad") cholesterol also was observed.
- Rheumatoid arthritis. Studies suggest fish oil supplements might help reduce pain, improve morning stiffness and relieve joint tenderness in people with rheumatoid arthritis. While relief is often modest, it might be enough to reduce the need for anti-inflammatory medications.

Our take
Green light: Generally safe
Generally safe
Omega-3 fatty acids are essential for good health. Try to get them from your diet by eating fish — broiled or baked, not fried. Fish oil supplements might be helpful if you have high triglycerides or rheumatoid arthritis.

Fish oil appears to contain almost no mercury, which can be a cause for concern in certain types of fish. While generally safe, getting too much fish oil can increase your risk of bleeding and might affect your immune response. It's not clear whether fish oil is safe for people who are allergic to seafood. Take fish oil supplements under a doctor's supervision.

Safety and side effects
When taken as recommended, fish oil supplements are generally considered safe.

However, fish oil supplements can cause mild side effects, including:
- A fishy aftertaste
- Bad breath
- Heartburn, nausea or diarrhea
- Rash
Taking high doses of fish oil supplements might increase the risk of bleeding and possibly increase the risk of stroke.
'''
},
{
    'title': 'Should I take Fish oil? - The New York Times', 
    'url': 'https://www.nytimes.com/2019/11/01/style/self-care/fish-oil-benefits.html',
    'text': '''Should I Take Fish Oil?
Omega-3 in fish oil is said to improve arthritis and reduce the likelihood of heart attacks. But the science suggests: Maybe just eat a fish once in a while.

In 1971, a team of Danish researchers found that an Inuit population in Greenland had lower cholesterol levels and incidence of heart disease than Danes and Inuits who lived in Denmark. A theory arose: The indigenous people’s high marine-fat diet gave them heart protections.

Since then, fish oil and omega-3 fatty acids — the health-boosting star component of fish oil — have been the subject of tens of thousands of papers exploring their health benefits.

Today, the “Eskimo theory” has been largely discredited. “We really don’t know if the Eskimos got heart disease or not,” said Malden C. Nesheim, an emeritus professor of nutrition at Cornell University, who chaired an Institute of Medicine committee assessing the risks and benefits of seafood in the early 2000s, in a 2018 interview with The New York Times. “I’ve been an omega-3 skeptic since doing this study.”

What do proponents of fish oil say it will do for you?
Fish oil is said to: improve arthritis; reduce ADHD; reduce the likelihood of heart attack and cancer; improve high-density lipoprotein (HDL, the so-called good) cholesterol.

Should I try fish oil supplements?
For healthy individuals, fish oil supplements are unnecessary. It’s better to eat a few servings of fish a week, instead.

The largest study — called the Vital study — done by Brigham and Women’s Hospital, an affiliate of Harvard Medical School, followed more than 25,000 people since 2010 and focused on whether taking daily dietary supplements of vitamin D or omega-3 fatty acids reduced the risk of cardiac events or cancer in otherwise healthy individuals.

It found that omega-3 supplements didn’t reduce the risk of major cardiac events in a usual-risk population, but did reduce the risk in a subset of people with low fish intake by 19 percent. The study is considered the medical gold standard.

African-Americans benefited regardless of fish intake, showing a 77 percent lower risk of heart attack. “This could be a chance finding,” said Dr. JoAnn Manson, a director of the study and the chief of the Division of Preventive Medicine at Brigham and Women’s Hospital. “We do plan to pursue it in greater detail and try to replicate it in a separate trial because if this can be reproduced, that would be a very dramatic benefit to African-Americans.”

Because there is still more research to be done, experts don’t necessarily recommended that African-Americans take omega-3.

If you have some history of heart disease or high triglycerides (an estimated 25 percent of adults in the United States do, according to data from the National Health and Nutrition Examination Survey in 2015), it may be a good idea to take omega-3.

The potential downside, because supplements are not regulated, is that production isn’t standardized so we don’t know what’s in them, according to Dr. Pieter Cohen, of Cambridge Health Alliance, who is an associate professor of medicine at Harvard Medical School.

He said that supplements are expensive and that money could alternatively be spent on a healthier diet. As an internist, Dr. Cohen has seen negative behavioral effects in some of his patients who take supplements.

“I have many patients who are like, ‘I’ll take my supplement and then I won’t worry about eating healthfully during the day,’” Dr. Cohen said. “That’s really misguided. Because in this case we have absolutely no evidence that replacing a healthy meal of fish with an omega-3 supplement is better.”
'''
},
{
    'title': '12 Benefits of Taking Fish Oil - Healthline',
    'url': 'https://www.healthline.com/nutrition/benefits-of-fish-oil', 

}
]

In [5]:
query = 'Should I take fish oil?'

In [ ]:
chatgpt_roots = guidance('''{{#system~}}
{{system_prompt}}
{{~/system}}
{{#user~}}
Please find the roots of the following equation: {{equation}}
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions.
{{/user}}
{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''', llm=chatgpt, system_prompt=chatgpt_system)
chatgpt_roots(equation=poly)
# program = program(system_prompt=chatgpt_system, equation=poly)

In [36]:
meeting_transcript = '''John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: And let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Lucy: Those are valid concerns, John, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
Steven: You know what Lucy, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: But why do you think it might change?
John: Well, just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: Right, I share this concern''' 

In [65]:
chatgpt_answer = guidance('''{{#system~}}
{{system_prompt}}
{{~/system}}
{{#user~}}
Here is a meeting transcript:
----
{{transcript}}
----
Question: {{query}}
Based on the above, please answer the following question:
Question: {{query}}
Please extract from the transcript whichever parts will support the answer, and then answer the question.
{{/user}}
{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''', llm=chatgpt, system_prompt=chatgpt_system)
p = chatgpt_answer(transcript=meeting_transcript, query='How does Steven feel about selling?', silent=False)
# Rather than answering the question, please extract from the transcript whichever parts will support the answer.


Stop program system You are a helpful assistant. user Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: And let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Lucy: Those are valid concerns, John, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
Steven: You know what Lucy, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: But why do you think it might change?
John: Well, just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: Right, I share this concern 
----
Question: How does Steven feel about selling? 
Rather than answering the question, please extract from the transcript whichever parts will support the answer.
 assistant Steven is in favor of selling the startup to Microsoft. 

Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.

Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.

Steven: And let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.

Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?

In [67]:
vicuna_answer = guidance('''{{#system~}}
{{system_prompt}}
{{~/system}}
{{#user~}}
Here is a meeting transcript:
----
{{transcript}}
----
Based on the above, please answer the following question:
Question: {{query}}
Please extract from the transcript whichever parts will support the answer, and then answer the question.
{{/user}}
{{#assistant~}}
Ok, first I will copy from the transcript the parts that will support my answer, and then I will answer the question.
Relevant parts:
{{gen 'rest' temperature=0}}
{{~/assistant~}}''', llm=vicuna, system_prompt=vicuna_system)
p = vicuna_answer(transcript=meeting_transcript, query='How does Steven feel about selling?', silent=False)
# {{gen 'quotes' pattern="(John|Steven|Lucy):.*" temperature=0 stop="\\n"}}


Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: And let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Lucy: Those are valid concerns, John, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
Steven: You know what Lucy, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: But why do you think it might change?
John: Well, just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: Right, I share this concern 
----
Based on the above, please answer the following question:
Question: How does Steven feel about selling? 
Please extract from the transcript whichever parts will support the answer, and then answer the question.
 assistant Ok, first I will copy from the transcript the parts that will support my answer, and then I will answer the question.
Relevant parts:
 
* "I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business."
* "And let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team."
* "But why do you think it might change? Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist."
* "I just don't want to lose what we've built here."

Answer: Steven is hesitant about selling the startup, he is not sure if the two companies' cultures can coexist, and he is also worried about losing what they have built.

In [10]:
chatgpt_answer = guidance('''{{#system~}}
{{system_prompt}}
{{~/system}}
{{#user~}}
Here are a few webpages from a websearch:
----
WEBPAGE 1
{{page.title}}
{{page.text}}
----
WEBPAGE 2
{{page2.title}}
{{page2.text}}
----
Based on the information above, please answer the following question:
Question: {{query}}
Make sure to attribute any factual claims to the appropriate source. For example, if a factual claim is supported by webpage 1, add [1] after the factual claim.
{{/user}}
{{#assistant~}}
{{gen 'answer'}}
{{~/assistant~}}''', llm=chatgpt, system_prompt=chatgpt_system)
p = chatgpt_answer(page=pages[0], page2=pages[1], query=query, silent=False)


Stop program system You are a helpful assistant. user Here are a few webpages from a websearch:
----
WEBPAGE 1
 Fish oil - Mayo Clinic 
 Overview
Fish oil is a dietary source of omega-3 fatty acids. Your body needs omega-3 fatty acids for many functions, from muscle activity to cell growth.

Omega-3 fatty acids are derived from food. They can't be manufactured in the body. Fish oil contains two omega-3s called docosahexaenoic acid (DHA) and eicosapentaenoic acid (EPA). Dietary sources of DHA and EPA are fatty fish, such as salmon, mackerel and trout, and shellfish, such as mussels, oysters and crabs. Some nuts, seeds and vegetable oils contain another omega-3 called alpha-linolenic acid (ALA).

Fish oil supplements come in liquid, capsule and pill form.

People take fish oil for its anti-inflammatory effects.

Evidence
Research on the use of fish oil for specific conditions shows:

- Heart disease. While research shows that people who eat dietary sources of fish oil at least twice a week have a lower risk of dying of heart disease, taking fish oil supplements seems to have little to no benefits to heart health.
- High blood pressure. Multiple studies report modest reductions in blood pressure in people who take fish oil supplements. There's some evidence that the beneficial effects of fish oil might be greater for people with moderate to severe high blood pressure than for those with mild blood pressure elevation.
- High triglycerides and cholesterol. There's strong evidence that omega-3 fatty acids can significantly reduce blood triglyceride levels. There also appears to be a slight improvement in high-density lipoprotein (HDL, or "good") cholesterol, although an increase in levels of low-density lipoprotein (LDL, or "bad") cholesterol also was observed.
- Rheumatoid arthritis. Studies suggest fish oil supplements might help reduce pain, improve morning stiffness and relieve joint tenderness in people with rheumatoid arthritis. While relief is often modest, it might be enough to reduce the need for anti-inflammatory medications.

Our take
Green light: Generally safe
Generally safe
Omega-3 fatty acids are essential for good health. Try to get them from your diet by eating fish — broiled or baked, not fried. Fish oil supplements might be helpful if you have high triglycerides or rheumatoid arthritis.

Fish oil appears to contain almost no mercury, which can be a cause for concern in certain types of fish. While generally safe, getting too much fish oil can increase your risk of bleeding and might affect your immune response. It's not clear whether fish oil is safe for people who are allergic to seafood. Take fish oil supplements under a doctor's supervision.

Safety and side effects
When taken as recommended, fish oil supplements are generally considered safe.

However, fish oil supplements can cause mild side effects, including:
- A fishy aftertaste
- Bad breath
- Heartburn, nausea or diarrhea
- Rash
Taking high doses of fish oil supplements might increase the risk of bleeding and possibly increase the risk of stroke.
 
----
WEBPAGE 2
 Should I take Fish oil? - The New York Times 
 Should I Take Fish Oil?
Omega-3 in fish oil is said to improve arthritis and reduce the likelihood of heart attacks. But the science suggests: Maybe just eat a fish once in a while.

In 1971, a team of Danish researchers found that an Inuit population in Greenland had lower cholesterol levels and incidence of heart disease than Danes and Inuits who lived in Denmark. A theory arose: The indigenous people’s high marine-fat diet gave them heart protections.

Since then, fish oil and omega-3 fatty acids — the health-boosting star component of fish oil — have been the subject of tens of thousands of papers exploring their health benefits.

Today, the “Eskimo theory” has been largely discredited. “We really don’t know if the Eskimos got heart disease or not,” said Malden C. Nesheim, an emeritus professor of nutrition at Cornell University, who chaired an 